In [2]:
from bs4 import BeautifulSoup
import sys
from urllib.parse import urlparse

In [18]:
if 'ipykernel' in sys.modules:
    sys.argv = [
        sys.argv[0],
        "https://fiddlersgreene.substack.com/p/the-dissident-right-and-its-discontents",
        "div",
        "available-content",
    ]

pathOrUrl = sys.argv[1]

if pathOrUrl.startswith('http'):
    parsed_url = urlparse(pathOrUrl)
    path = parsed_url.path

    # Split the path by slashes and take the last element
    if path[-1] == '/':
        path = path[:-1]

    output_file_stem = path.split('/')[-1]

    # Fetch the HTML content
    import requests
    response = requests.get(pathOrUrl)
    html_content = response.text
else:
    from pathlib import Path
    output_file_stem = Path(pathOrUrl).stem
    with open(pathOrUrl) as f:
        html_content = f.read()


In [19]:
# Parse the HTML content with Beautiful Soup
soup = BeautifulSoup(html_content, 'html.parser')

article = soup.find(sys.argv[2]) if len(sys.argv) < 3 else soup.find(sys.argv[2], class_=sys.argv[3])
# article = soup.find(sys.argv[2], class_=sys.argv[3])

if not article:
    print("no atricle element")
    body = soup.body
    # for header in body.find_all('script'):
    #     header.decompose()
    
    # for script in body.find_all('script'):
    #     script.decompose()

    # for svg in body.find_all('svg'):
    #     svg.decompose()

    print(body)
    print(sys.argv[1:])
    exit(1)

# Find and remove all <code> elements
for code_tag in article.find_all('pre'):
    code_tag.decompose()

# Extract text and remove leading/trailing whitespace
text_content = article.get_text(separator='\n', strip=True)
print(text_content)

The Dissident Right
A specter is haunting the internet; its name is the dissident right.
In this case, the term "specter" is rather literal since the dissident right was dead before it even became a thing, if it ever was a thing. I first remember the term coming into prominence in the wake of the rapid rise and fall of the original "Alt-Right", as people needed some other word to describe right-wingers who were neither normie-cons nor libertarians.
For this reason, I frequently employed the term "dissident right" to describe myself between 2017 and 2021, even though I never thought it meant much of anything beyond describing things I didn't believe.
What did it mean to be “dissident right”?
Did it mean NOT progressive?
Did it mean NOT a “LOLbertarian”?
Did it mean NOT a David-French-style normie con?
Skeptical of mainstream power, either state or corporate?
Sure, that makes me a "dissident", so the name dissident right fits well enough.
But the term was contentious, even if it was alwa

In [6]:
from nltk.data import load
from TTS.api import TTS
tts = TTS(model_name="tts_models/en/vctk/vits", gpu=True)
sample_rate = tts.synthesizer.output_sample_rate

In [7]:
tokenizer = load(f"tokenizers/punkt/english.pickle")
sentences = tokenizer.tokenize(text_content)
print(sentences)

['In Support of Implication for C++ – Why is a raven like a writing desk?', 'Skip to content\nWhy is a raven like a writing desk?', 'Thoughts both confusing and enlightening.', 'In Support of Implication for C++\nThe paper:\nhttps://wg21.link/p2971\nI have a deck of cards.', 'Each card has a number (integer) on the front, and is either red or blue on the back.', '(This is axiomatic, I know it\na priori\n).', 'My friend lays out four cards on the table in front of us at random.', 'I see two face up, showing 3 and 8, and two face down, showing a red back and a blue back.', '(Image from https://en.wikipedia.org/wiki/File:Wason_selection_task_cards.svg, CC by-sa 4.0)\nMy friend makes a claim: “if a card has an even number on the face, it has a blue back.” Which cards do I need to turn over to test this claim?', 'This is the famously unintuitive\nWason Selection Task\n.', 'It’s answered incorrectly by the majority of people, even people trained in logic (math students, graduates, and yes, p

In [13]:
import subprocess
from TTS.tts.utils.synthesis import synthesis

command = [
            'ffmpeg', '-y',
            '-f', 'f32le', 
            '-ar', str(sample_rate),  # sample rate
            '-ac', '1',  # number of audio channels
            '-i', '-',  # The input comes from stdin
            '-acodec', 'copy',  # audio codec for M4A
            output_file_stem + '.wav',
        ]
print(" ".join(command))

process = subprocess.Popen(command, stdin=subprocess.PIPE)

for sentence in sentences:
    wav = synthesis(
            model=tts.synthesizer.tts_model,
            text=sentence,
            CONFIG=tts.synthesizer.tts_config,
            use_cuda=True,
            speaker_id=77, # p307
            use_griffin_lim=tts.synthesizer.vocoder_model is None,
            )["wav"]
    format = wav
    process.stdin.write(wav.tobytes())

process.stdin.close()
process.wait()
print("------------\n", format)

ffmpeg -y -f f32le -ar 22050 -ac 1 -i - -acodec copy blog.wav


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

------------
 [-0.00024886 -0.00042764 -0.00031147 ...  0.00055181  0.00037304
  0.00013643]


size=   27012kB time=00:05:13.60 bitrate= 705.6kbits/s speed=6.31x    
video:0kB audio:27012kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.000412%


In [19]:
print(sys.argv)

['/home/a/.local/lib/python3.10/site-packages/ipykernel_launcher.py', '--f=/home/a/.local/share/jupyter/runtime/kernel-v2-17921kGKUpKVYOTJo.json']
